In [6]:
# 오페라 transaction code를 추출하는 모듈

import pandas as pnds
import xml.etree.ElementTree as et
import datetime
#from IPython.display import display

# 오페라 trial balance가 xml 파일로 저장됨
xtree = et.parse('./cf_trxcodes18359637.xml')
xroot = xtree.getroot()

# 실제 필요한 node(G_TRX_CODE 들) 추출
g_trx_codes = xtree.findall('LIST_G_TC_GROUP/G_TC_GROUP/LIST_G_TC_SUBGROUP/G_TC_SUBGROUP/LIST_G_TRX_CODE/G_TRX_CODE')

# trial balance용 dataframe 만들기
rows = []                       # index buffer
for node in g_trx_codes:        # each index를 looping
    cols = {}                   # column용 딕셔너리 buffer
    for item in node:           # index 내의 each column을 looping
        key = item.tag.strip()      # column용 딕셔너리 key
        cols.update({key: item.text})   # 원하는 column용 딕셔너리 추출

    rows.append(cols)           # 완성된 index 한 줄을 리스트에 추가

# 추출된 index 리스트에서 DataFrame 생성
card_df = pnds.DataFrame(rows)
card_df = card_df[['TRX_CODE', 'D3']]                       # transaction code와 그 description만 추출
card_df = card_df.rename(columns={'D3': 'Description'})     # description 컬럼명으로 변경
card_df = card_df[card_df['TRX_CODE'] >= '9231']            # 매뉴얼 카드 코드만 추출, TRX_CODE 컬럼은 문자열 데이터

# csv 파일로 저장
base_dir = r'C:\work\pycard\dtdata'
file_name = '/trx_card_codes.csv'
data_dir = base_dir + file_name
card_df.to_csv(data_dir, sep='\t', index=False)